In [ ]:
#notebook by Visarg,Avil and Aryan

In [ ]:
#Training code for task1 and task2(trained on task1)

In [ ]:
#The output will be Scores with the corresponding episode number and the training log(graph of Episodes v/s Scores)

In [ ]:
# --versions--
# numpy==1.18.2
# gym==0.17.1
# Keras==2.3.1
# Keras-Applications==1.0.8
# Keras-Preprocessing==1.1.0
# tensorboard==2.1.1
# tensorflow==2.1.0
# tensorflow-estimator==2.1.0
# matplotlib==3.2.1

In [ ]:
import numpy as np 
import gym 
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam  
import random
import matplotlib.pyplot as plt
from statistics import mean

In [ ]:
class Agent(): 
    
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        self.state_size = self.env.observation_space.shape[0] 
        self.action_size = self.env.action_space.n 
        
        self.gamma = 0.95
        self.learning_rate = 0.001
        self.batch_size = 20
        
        self.epsilon = 1 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.memory = deque(maxlen = 1200) 
        
        self.model = self.build_model()
    
    def build_model(self):
        model = Sequential()
        model.add(Dense(48, input_dim = self.state_size, activation = 'tanh')) 
        model.add(Dense(24,activation = 'tanh')) 
        model.add(Dense(self.action_size, activation = 'linear')) 
        model.compile(loss = 'mse', optimizer = Adam(lr = self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if random.uniform(0,1) <= self.epsilon:
            return self.env.action_space.sample()
        else:
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])
              
    def replay(self):
        if len(self.memory) < self.batch_size:
            return 
        
        minibatch = random.sample(self.memory, self.batch_size) 
        for state, action, reward, next_state, done in minibatch:
            if done: 
                target = reward
            else:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0]) 
                
            train_target = self.model.predict(state)
            train_target[0][action] = target
            self.model.fit(state, train_target, verbose = 0) 
    
    def exploration(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def train(self):
        episodes=[]
        final_scores=[]
        for e in range(500): # maximum episodes to train = 500
            
            state = self.env.reset() #reseting env for each episode
            state = np.reshape(state, [1,4])
            
            score = 0 #initialising score for each episode
            done = False
            
            while not done:
                action = self.act(state) #choosing action according to the state
                next_state, reward, done, _ = self.env.step(action) #taking action from above step
                next_state = np.reshape(next_state, [1,4]) 
                self.remember(state, action, reward, next_state, done) #pushing info in deque
                state = next_state #updating state
                self.exploration() #updating epsilon
                score+=1 #incrementing score as the reward for survival is 1
                
                if done: #when game(episode) is over
                    final_scores.append(score)
                    episodes.append(e+1)
                    print("Episode: {}, Score: {}".format(e+1, score))
                    if score == 500: # Stoping the training when score reaches 500 for the first time
                        print("Number of episodes to train : {}".format(e+1))
                        print("--Saving trained model--")
                        self.model.save("model_t1t2.h5") #saving model
                        #Training logs
                        plt.figure(1) 
                        plt.plot(episodes,final_scores)
                        plt.xlabel('Episodes')
                        plt.ylabel('Scores')
                        plt.title('Training')
                        plt.show()
                        return
                self.replay() #updating weights of NN for better prediction
                            

In [ ]:
agent = Agent()
agent.train()